In [73]:
vocab = {0: b' ', 1: b'a', 2: b'c', 3: b'e', 4: b'h', 5: b't',
         6: b'th', 7: b' c', 8: b' a', 9: b'the', 10: b' at'}

b_to_i = {v:k for k, v in vocab.items()}

merges = [(b't', b'h'), (b' ', b'c'), (b' ', b'a'), (b'th', b'e'), (b' a', b't')]

input = "the cat ate"

In [74]:
import regex as re

def encode(input, b_to_i, merges):
    """Encodes a string into a sequence of tokens"""

    pretokenized = re.findall(r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""", input)
    encoded_str = []

    # Encode each word independently
    for w in pretokenized:
        # Convert a word to a sequence of bytes wrapped in a list
        w = [bytes([ord(c)]) for c in w] # 'the' -> [b't', b'h', b'e']
        
        # Start merges
        for merge in merges:
            for idx, (b1, b2) in enumerate(zip(w, w[1:])):
                # If a match was found
                if merge == (b1, b2):
                    # Merge the bytes
                    w = [b1 + b2] + w[idx+2:]
                    # Breakout to update the merged string
                    break
        
        # Encode final merges to ints
        encoded_str.extend([b_to_i[b] for b in w])
    
    return encoded_str

In [78]:
ids = encode(input, b_to_i, merges)
ids

[9, 7, 1, 5, 10, 3]

In [76]:
def decode(token_ids, vocab):
    """Decodes a sequence of tokens to a string"""

    decoded_bytes = [vocab[id] for id in token_ids] # Map from integers to bytes
    decoded_str = ""
    
    # Decode into UTF-8 codec
    for byte in decoded_bytes:
        s = byte.decode("utf-8", errors="replace")
        # Concat the strings
        decoded_str += s
    return decoded_str

In [77]:
print(decode(ids, vocab))

the cat ate
